함수 메모장
파일 불러오기 : pd.read_csv('./tmdb_5000_credits.csv')
상위 n개 행 불러오기 : df2.head(n)
평균 : df2['행이름'].mean()
상위 100n% 값들만 필터링하기 : df2['행이름'].quantile(1 - n)
특정 행 검색 : df2.loc[조건식] (조건식 예시 : df2['행이름'] >= m)
특정 열 검색 : df2.loc['행이름']
각 행에 함수 적용 : q_movies.apply(함수명) (함수명 : 첫번째 parameter가 q_movies이여야함?)
행 정렬 : q_movies.sort_values(열name, ascending=False)



In [17]:
input = 'Horror Crime Thriller'

genres_list = ['Animation', 'Comedy', 'Family', 'Adventure', 'Fantasy', 'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror', 'History', 'Science_Fiction',
 'Mystery War', 'Foreign', 'Music', 'Documentary', 'Western', 'TV_Movie', 'Carousel_Productions', 'Vision_View_Entertainment', 'Telescene_Film_Group_Productions', 
 'Aniplex GoHands', 'BROSTA_TV', 'Mardock_Scramble_Production_Committee', 'Sentai_Filmworks', 'Odyssey_Media' ,'Pulser_Productions', 'Rogue_State', 'The_Cartel']
genres_infor = 'Animation Comedy Family Adventure Fantasy Romance Drama Action Crime Thriller Horror History Science_Fiction Mystery War Foreign Music Documentary Western TV_Movie Carousel_Productions Vision_View_Entertainment Telescene_Film_Group_Productions Aniplex GoHands BROSTA_TV Mardock_Scramble_Production_Committee Sentai_Filmworks Odyssey_Media Pulser_Productions Rogue_State The_Cartel'
# 32개의 장르

from ast import literal_eval
import pandas as pd
import numpy as np
content = pd.read_csv('lingo_ex_movie_small.csv')

content.head(3)

,Unnamed: 0,belongs_to_collection,genres,id,production_companies,release_date,title,vote_average,vote_count,series_num
0,1039,NaN,"['Crime', 'Drama', 'Mystery', 'Thriller']",1834,RKO Radio Pictures,1944-12-09,"Murder, My Sweet",7.2,48.0,0
1,1133,NaN,"['Crime', 'Drama']",7452,Elsevira,1989-09-10,"The Cook, the Thief, His Wife & Her Lover",7.3,137.0,0
2,1682,NaN,['Comedy'],66894,Dimension Films,1998-03-27,Ride,6.8,4.0,0


In [18]:
content['genres'] = content['genres'].apply(literal_eval)
content['genres_literal'] = content['genres'].apply(lambda x : (' ').join(x))
content['genres_literal'].head(10)

0                         Crime Drama Mystery Thriller
1                                          Crime Drama
2                                               Comedy
3                                       Drama Thriller
4    Thriller Adventure Fantasy Science_Fiction Rom...
5                                               Horror
6                                        Comedy Horror
7                                                Drama
8                          Adventure Drama Romance War
9                                               Comedy
Name: genres_literal, dtype: object

In [19]:
genres_list = []
for i in range(len(content)):
    gen_list = content.loc[i,'genres']
    for j in gen_list:
        if j not in genres_list:
            genres_list.append(j)
print(genres_list)
genres_infor = ''
for i in genres_list:
    genres_infor += i + ' '
print(genres_infor)

['Crime', 'Drama', 'Mystery', 'Thriller', 'Comedy', 'Adventure', 'Fantasy', 'Science_Fiction', 'Romance', 'Horror', 'War', 'Documentary', 'Action', 'History', 'TV_Movie', 'Family', 'Animation', 'Foreign']
Crime Drama Mystery Thriller Comedy Adventure Fantasy Science_Fiction Romance Horror War Documentary Action History TV_Movie Family Animation Foreign 


In [20]:
# 모든 변수가 텍스트로 이루어졌으니 Countvectorizer로 feature 벡터화시키기
# 경우에 따라 추가적으로 Lemmatization 추가해 토큰화 전용 함수 추가도 가능!

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

input_vect = TfidfVectorizer().fit_transform(pd.Series([input, genres_infor]))
genres_vect = TfidfVectorizer().fit_transform(content['genres_literal'])

# 장르에 따른 콘텐츠별 코사인 유사도 추출
genre_sim = cosine_similarity(input_vect, genres_vect)

In [21]:

def sort_similar(input):
    for i in range(len(input)):
        input[i] = input[i].argsort()[::-1]
sort_similar(genre_sim)
print(genre_sim)


[[18. 43. 16.  0.  5.  1.  6. 28. 46.  3. 10. 22. 25. 49. 34. 38. 11.  4.
   9. 15. 14. 13. 12. 41. 35. 40.  7. 42. 44. 45.  2. 47.  8. 17. 39. 19.
  20. 21. 23. 48. 37. 26. 27. 36. 29. 30. 31. 32. 33. 24.]
 [41.  4. 11.  8. 20. 44. 13. 33. 16. 36. 23.  0. 43. 35. 24. 49. 25. 38.
  42. 29. 39. 12. 18. 40. 17. 34. 22. 10.  3. 28. 46.  6.  1.  2.  7.  9.
  48. 31. 14. 15. 19. 27. 21. 32.  5. 26. 45. 30. 37. 47.]]


In [22]:

#print(input_sim_idx[0])
answer = []
answer2 = []
for i in range(10):
    answer.append(content.loc[genre_sim[0][i]][['title']])
    answer2.append(content.loc[genre_sim[0][i]][['genres']])
print(answer)
print(answer2)




[title    Man on a Ledge
Name: 18, dtype: object, title    Dossier K.
Name: 43, dtype: object, title    The Guard
Name: 16, dtype: object, title    Murder, My Sweet
Name: 0, dtype: object, title    Popcorn
Name: 5, dtype: object, title    The Cook, the Thief, His Wife & Her Lover
Name: 1, dtype: object, title    The Lair of the White Worm
Name: 6, dtype: object, title    In the Eyes of the World
Name: 28, dtype: object, title    Kóblic
Name: 46, dtype: object, title    In Dreams
Name: 3, dtype: object]
[genres    [Action, Thriller, Crime]
Name: 18, dtype: object, genres    [Action, Crime, Drama, Thriller]
Name: 43, dtype: object, genres    [Action, Comedy, Thriller, Crime]
Name: 16, dtype: object, genres    [Crime, Drama, Mystery, Thriller]
Name: 0, dtype: object, genres    [Horror]
Name: 5, dtype: object, genres    [Crime, Drama]
Name: 1, dtype: object, genres    [Comedy, Horror]
Name: 6, dtype: object, genres    [Thriller, Drama]
Name: 28, dtype: object, genres    [Drama, Thriller]
N